In [11]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import date, timedelta
import datetime
from datetime import datetime

In [12]:
time.sleep(2)
def get_links(url, div):
    page=requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    soup2 = soup.find_all('div', class_=div)
    links = ['https://econs.online' + s.find('a')['href'] for s in soup2]
    titles = [s.find('a').text for s in soup2]
    return pd.DataFrame({'link': links, 'title': titles})
def get_text(url):
    articles = requests.get(url)
    articles_soup = BeautifulSoup(articles.text, "html.parser")
    articles_content = articles_soup.find_all('p')
    articles_text = [paragraph.get_text() for paragraph in articles_content]
    return ' '.join(articles_text)
def get_date(url):
    dates = requests.get(url)
    dates_soup = BeautifulSoup(dates.text, "html.parser")
    dates_info = dates_soup.find('time')['datetime'][:10]
    return ''.join(dates_info)
div = 'news-card__title'
more = 84
df_ekons = pd.DataFrame({'link': [], 'title': [], 'text': [], 'date': []})

url = 'https://econs.online/?more='+ str(more)
df = get_links(url, div)
content = [get_text(url) for url in df['link'].tolist()]
df['text'] = content
date = [get_date(url) for url in df['link'].tolist()]
df['date'] = date
df_ekons = pd.concat([df_ekons, df], axis=0)

In [13]:
df_ekons

,link,title,text,date
0,https://econs.online/articles/ekonomika/dedush...,\n «Дедушка» поведенчес...,\r\n\t Даниэль Канеман не был экономистом и не...,2024-03-29
1,https://econs.online/articles/ekonomika/prosto...,\n Простой расчет: как ...,\r\n\t Действующие в разных странах схемы упро...,2024-03-27
2,https://econs.online/articles/opinions/yarmark...,\n Ярмарка щедрости: гл...,\r\n\t Как дивиденды влияют на стоимость компа...,2024-03-25
3,https://econs.online/articles/coffee-break/pom...,\n Помнить не все: чем ...,\r\n\t Человеческий мозг способен хранить инфо...,2024-03-22
4,https://econs.online/articles/ekonomika/tranzh...,"\n «Транжиры», «самостр...",\r\n\t Как люди воспользуются материальными ст...,2024-03-20
...,...,...,...,...
685,https://econs.online/articles/finansy/orel-ili...,\n Орел или решка: поче...,"\r\n\t Однажды Бертон Малкиел, профессор эконо...",2020-12-29
686,https://econs.online/articles/blogs/rastsvet-n...,"\n Расцвет науки, иннов...","\nПрогресс в науке, достигнутый за 2020 г., ст...",2020-12-28
687,https://econs.online/articles/coffee-break/uda...,\n Удаленка плюс офис: ...,\r\n\t Карантин и социальное дистанцирование п...,2020-12-25
688,https://econs.online/articles/ekonomika/vyzovy...,\n Вызовы экономики Кит...,\r\n\t После начала пандемии коронавируса экон...,2020-12-24


In [ ]:
df_ekons['date'] = pd.to_datetime(df_ekons['date'], format='%Y-%m-%d')
df_ekons['date'] = df_ekons['date'].dt.strftime('%d.%m.%Y')

In [15]:
df_ekons['source'] = 'Эконс'

In [17]:
df_ekons = df_ekons.replace(r'\t|\n|\r|\s+',' ', regex=True)
for col in df_ekons.select_dtypes('object'):
    df_ekons[col] = df_ekons[col].str.strip()

In [18]:
df_ekons.to_csv(f'C:\\Users\\oksik\\Desktop\\Ekons_done.csv', 
             index=False, sep='\t', encoding='utf-8', header=True, columns=["link","title","text","date","source"])